# Importing libraries.

In [1]:
import pandas as pd, numpy as np, seaborn as sns, ipywidgets as widgets

from ipywidgets import Layout

# Acquiring data.

In [2]:
df_traffic = pd.read_csv("traff_ga.csv", delimiter = ";")
df_channels = pd.read_csv("Каналы.csv")
df_rk = pd.read_csv("rk.csv")
df_sales = pd.read_csv("sales.csv")

# Data engineering.

In [3]:
df_traffic.set_index("ga_user_id", inplace = True)
df_traffic["Channel"] = None

df_channels.set_index("Канал трафика", inplace = True)

df_rk.set_index("Канал трафика", inplace = True)
df_rk['Total'] = df_rk.sum(axis = 1)

df_sales.set_index("Номер заявки", inplace = True)
df_sales["Стоимость услуги"].astype(float)
df_sales["Сумма НДС"].astype(float)
df_sales["Сумма НДС"].fillna(value = 0, inplace = True)
df_sales["Revenue"] = df_sales["Стоимость услуги"] - df_sales["Сумма НДС"]

# Mapping.
## df_traffic.

In [4]:
df_traffic.loc[df_traffic["source"].isin(["yandex", "yandex-search", "yandex-network", "direct", "yandex-direct", "Yandex"]), "Channel"] = "Я.Директ,р"

df_traffic.loc[df_traffic["source"].isin(["vkontakte", "Telegram", "etp_roseltorg"]), "Channel"] = "VK, таргетинг,р"

df_traffic.loc[(df_traffic["source"].isin(["mindbox", "email", "emarsys"])) | (df_traffic["medium"].isin(["mail"])), "Channel"] = "Email-рассылка, сервис,р"

df_traffic.loc[(df_traffic["source"].isin(["sms"])) | (df_traffic["medium"].isin(["CoMagic"])), "Channel"] = "whatsapp, рассылка,р"

df_traffic.loc[df_traffic["medium"].isin(["organic"]), "Channel"] = "СЕO, контенкт,р"

df_traffic.loc[df_traffic["source"].isin(["yandex.promopages", "dzen_suggest_ad_priority", "dzen", "dzen_suggest_nav_priority"]), "Channel"] = "Я.Дзен,р"

df_traffic.loc[df_traffic["medium"].isin(["referral"]), "Channel"] = "Реферальный трафик"

df_traffic.loc[df_traffic["source"].isin(["push", "posteruc", "qr", "QR", "vebinar",  "booklet"]), "Channel"] = "Промо-материалы,р"

df_traffic.loc[df_traffic["source"].isin(["gos.roseltorg.ru", "com.roseltorg.ru", "178fz", "flocktory", "partner", "etp_rosltorg", "etp-roseltorg", "com_rosltorg", "roseltorg_ep", "rostec.roseltorg.ru", "com_roseltorg"]), "Channel"] = "Партнерский маркетинг,р"

#df_traffic.loc[df_traffic["source"].isin(["(direct)"]), "Channel"] = "Прямая ссылка"

## df_sales.

In [5]:
df_sales.loc[df_sales["utm_source"].isin(["yandex", "yandex-search", "yandex-network", "direct", "yandex-direct", "Yandex"]), "Channel"] = "Я.Директ,р"

df_sales.loc[df_sales["utm_source"].isin(["vkontakte", "Telegram", "etp_roseltorg"]), "Channel"] = "VK, таргетинг,р"

df_sales.loc[(df_sales["utm_source"].isin(["mindbox", "email", "emarsys"])) | (df_sales["utm_medium"].isin(["mail"])), "Channel"] = "Email-рассылка, сервис,р"

df_sales.loc[(df_sales["utm_source"].isin(["sms"])) | (df_sales["utm_medium"].isin(["CoMagic"])), "Channel"] = "whatsapp, рассылка,р"

df_sales.loc[df_sales["utm_medium"].isin(["organic"]), "Channel"] = "СЕO, контенкт,р"

df_sales.loc[df_sales["utm_source"].isin(["yandex.promopages", "dzen_suggest_ad_priority", "dzen", "dzen_suggest_nav_priority"]), "Channel"] = "Я.Дзен,р"

df_sales.loc[df_sales["utm_medium"].isin(["referral"]), "Channel"] = "Реферальный трафик"

df_sales.loc[df_sales["utm_source"].isin(["push", "posteruc", "qr", "QR", "vebinar",  "booklet"]), "Channel"] = "Промо-материалы,р"

df_sales.loc[df_sales["utm_source"].isin(["gos.roseltorg.ru", "com.roseltorg.ru", "178fz", "flocktory", "partner", "etp_rosltorg", "etp-roseltorg", "com_rosltorg", "roseltorg_ep", "rostec.roseltorg.ru", "com_roseltorg"]), "Channel"] = "Партнерский маркетинг,р"

#df_sales.loc[df_sales["utm_source"].isin(["(direct)"]), "Channel"] = "Прямая ссылка"

# Metrics.
## Traffic.

In [6]:
df_count = pd.DataFrame(df_traffic.groupby(by = ["Channel"])["fullReferrer"].count())

## Revenue.

In [7]:
df_revenue = pd.DataFrame(df_sales.groupby(["Channel"])["Revenue"].sum())

## Average receipt

In [8]:
df_average_receipt = df_revenue.join(df_count, how = "inner")
df_average_receipt["Average receipt"] = df_average_receipt["Revenue"] / df_average_receipt["fullReferrer"]
df_average_receipt.drop(["Revenue", "fullReferrer"], axis = 1, inplace = True)

## Conversion rate.

In [9]:
df_va = pd.DataFrame(df_sales.groupby(["Channel"])["Revenue"].count())

df_cr = df_va.join(df_count, how = "inner")
df_cr["CR"] = df_cr["Revenue"] / df_cr["fullReferrer"] * 100
df_cr.drop(["Revenue", "fullReferrer"], axis = 1, inplace = True)

## ROI.

In [10]:
df_total_cost = pd.DataFrame(df_rk["Total"])

df_roi = df_revenue.join(df_total_cost, how = "inner")
df_roi["ROI"] = (df_roi["Revenue"] - df_roi["Total"]) / df_roi["Total"]
df_roi.drop(["Revenue", "Total"], axis = 1, inplace = True)

# Creating an interactive dashboard (a try).

In [11]:
df_merged = df_revenue.merge(df_average_receipt, left_index = True, right_index = True).merge(df_cr, left_index = True, right_index = True)
df_merged = df_merged.join(df_count, how = "outer")
df_merged = df_merged.join(df_roi, how = "outer")
df_merged.rename(columns = {"fullReferrer": "Traffic"}, inplace = True)
df_merged.fillna(value = 0, inplace = True)

In [12]:
unique_features = df_merged.columns.unique().to_numpy()

feature = widgets.Select(options = unique_features, value = "Revenue", description = 'Feature',
                         disabled = False, layout = Layout(width = '50%', height= '80px', display = 'flex'))

def graph(feature):
    df_feature = df_merged[feature]
    sns.barplot(x = df_merged[feature], y = df_merged.index)
    
widgets.interactive(graph, feature = feature)

interactive(children=(Select(description='Feature', layout=Layout(display='flex', height='80px', width='50%'),…